### ddrobot VSP version

Use with Wokwi link : https://wokwi.com/projects/397298628049688577

In [1]:
# class implementation of robo2links controller GUI
import numpy as np
import tkinter as tk
from tkinter import *
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle 
from matplotlib.animation import FuncAnimation
import serial
import time

In [2]:
# use code from this link
# https://stackoverflow.com/questions/60413174/rotating-rectangles-around-point-with-matplotlib

In [3]:
class RotatingRectangle(Rectangle):
    def __init__(self, xy, width, height, rel_point_of_rot, **kwargs):
        super().__init__(xy, width, height, **kwargs)
        self.rel_point_of_rot = rel_point_of_rot
        self.xy_center = self.get_xy()
        self.set_angle(self.angle)

    def _apply_rotation(self):
        angle_rad = self.angle * np.pi / 180
        m_trans = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                            [np.sin(angle_rad), np.cos(angle_rad)]])
        shift = -m_trans @ self.rel_point_of_rot
        self.set_xy(self.xy_center + shift)

    def set_angle(self, angle):
        self.angle = angle
        self._apply_rotation()

    def set_rel_point_of_rot(self, rel_point_of_rot):
        self.rel_point_of_rot = rel_point_of_rot
        self._apply_rotation()

    def set_xy_center(self, xy):
        self.xy_center = xy
        self._apply_rotation()

In [4]:
# ----- wheel parameters -------
# LEFT_WHEEL = 0
# RIGHT_WHEEL = 1
# WHEEL_RADIUS = 0.08
# WHEEL_DIAMETER = 2*WHEEL_RADIUS
# WHEEL_THICKNESS = 0.04
# WHEEL_SEPARATION = 0.45
# BASE_LENGTH = 0.65
# BASE_WIDTH = 0.55
# EYE_THICKNESS = 0.04
# EYE_DIAMETER = 0.1


In [5]:
# def rotate_points(angle,xy,center):
#     angle_rad = angle * np.pi / 180
#     m_trans = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
#                         [np.sin(angle_rad), np.cos(angle_rad)]])
#     shift = m_trans @ xy    
#     return np.add(shift,center)
    

In [ ]:
# GUI for differential-drive robot. VSP version
class ddrControllerVSP:

    def __init__(self, root):
        
        screen_width = 1200
        screen_height = 1000
        self.TFont1 = ("Consolas",14)
        self.TFont2 = ("Consolas",16)
#         self.TFont1 = ("Calibri",14)
#         self.TFont2 = ("Calibri",16)
        
#         self.online = True  # change to True to connect to NETPIE
        self.rad2deg = 180/np.pi
        
        root.title("Differential-Drive Robot Controller (VSP version)")
        root.protocol("WM_DELETE_WINDOW", self.on_close)
        root.geometry(str(screen_width)+'x'+str(screen_height))
        mainframe = ttk.Frame(root, padding="3 3 12 12")
        mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
        root.columnconfigure(0, weight=1)
        root.rowconfigure(0, weight=1)
        cvrowspan = 15  # canvas row and column span.
        cvcolspan = 8  
        self.vspon = True  # flag whether to use VSP or not

        #---- serial communication setup ------------
        if self.vspon:
            self.ser = serial.Serial('COM8', 115200, timeout=1)
            self.ser.reset_input_buffer() # flush all data that you don't wanna read
            self.ser.reset_output_buffer()
        self.msgline = []
        self.msglines = []        
        
        

        self.enable_txt = StringVar()
        self.xg_txt = StringVar()   # goal XY coordinate 
        self.yg_txt = StringVar()
        self.xt_txt = StringVar()  # XY coordinate of robot
        self.yt_txt = StringVar()
        self.theta_txt = StringVar() # orientation of robot
        self.trackmsg_txt = StringVar()
        self.trackmsg_txt.set("Ready")
        self.streaming_txt = StringVar()
        
        
        # send command to robot controller
        self.ddr_txt = StringVar()
        ddrcmd_entry = ttk.Entry(mainframe, textvariable=self.ddr_txt)
        ddrcmd_entry.grid(column=0, row=cvrowspan+5,sticky=(W, E))
        ttk.Button(mainframe, text="Send", command=self.send_ddr).grid(column=1, row=cvrowspan+5, sticky=W)

        
       
        #   ------ robot enable group.  ------------------------------
        enframe = ttk.Frame(mainframe, padding = (2,2,2,2),relief="sunken")
        enframe.grid(column=cvcolspan, row=0,sticky=(W)) #column=cvcolspan,row=0)        
        # self.feedback_txt.set("1")  
        ttk.Label(enframe, text="Robot          ", 
                  font=self.TFont1,foreground='maroon').grid(column=0, row=0, sticky=(N,W))        
        self.disable_rb = (ttk.Radiobutton(enframe,text="Disable", variable=self.enable_txt,
                                     value="0",command=self.send_cmd_disable).
                                     grid(column=0, row=1, sticky=W))
        self.enable_rb = (ttk.Radiobutton(enframe, text="Enable", variable=self.enable_txt,
                                     value="1",command=self.send_cmd_enable).
                                     grid(column=0, row=2, sticky=W))

        # adjust linear/angular velocity gains
        
        self.vk = 1.0
        self.wk = 3.0
        
        self.vk_txt = StringVar()
        self.wk_txt = StringVar()
        
        self.vk_txt.set(str(self.vk))
        self.wk_txt.set(str(self.wk))
        
        ttk.Label(enframe, text="Controller gains ", 
                  font=self.TFont1,foreground='maroon').grid(column=0, columnspan=2,
                    row=3, sticky=(N,W))
        # linear velocity scale
        ttk.Label(enframe, text="V ",font=self.TFont1).grid(column=1, 
                                        row=4, sticky=W)
        # note that tk scale is used instead of ttk here
        self.vgain_scale =(tk.Scale(enframe, orient=HORIZONTAL,length=150, 
                                    from_=1.0, to=5.0,
                        resolution=1.0, troughcolor="blue",
                                    variable=self.vk_txt))        
        self.vgain_scale.grid(column=0, row=4,sticky=(W))
        # this is how the slider is executed only on release!
        self.vgain_scale.bind("<ButtonRelease-1>", self.send_cmd_vk)  
        
        # angular velocity scale
        ttk.Label(enframe, text="W ",font=self.TFont1).grid(column=1, 
                        row=5, sticky=W)
        # note that tk scale is used instead of ttk here
        self.wgain_scale =(tk.Scale(enframe, orient=HORIZONTAL,length=150,
                                    from_=1.0, to=5.0,
                        resolution=1.0, troughcolor="green",
                                    variable=self.wk_txt))        
        self.wgain_scale.grid(column=0, row=5,sticky=(W))
        # this is how the slider is executed only on release!
        self.wgain_scale.bind("<ButtonRelease-1>", self.send_cmd_wk)        


        
        #  -------- cartesian coordinate frame ------------------
        
        xyframe = ttk.Frame(mainframe, padding = (2,2,2,2),relief="sunken")
        xyframe.grid(column=cvcolspan,row=3,sticky=(W))
        ttk.Label(xyframe, text="XY coordinates", 
                  font=self.TFont1,foreground='green').grid(column=0, row=0, sticky=W)
        ttk.Label(xyframe, text="X goal: ", 
                  font=self.TFont1,foreground='blue').grid(column=0, row=1, sticky=W)
        ttk.Label(xyframe, textvariable=self.xg_txt, 
                  font=self.TFont1,foreground='teal').grid(column=1, row=1, sticky=W)
        ttk.Label(xyframe, text="Y goal: ", 
                  font=self.TFont1,foreground='blue').grid(column=0, row=2, sticky=W)
        ttk.Label(xyframe, textvariable=self.yg_txt, 
                  font=self.TFont1,foreground='teal').grid(column=1, row=2, sticky=W)        
        ttk.Label(xyframe, textvariable=self.trackmsg_txt, 
                  font=self.TFont1,foreground='olive').grid(column=0, row=3, sticky=W)        
        
        ttk.Label(xyframe, text="X robot: ", 
                  font=self.TFont1,foreground='maroon').grid(column=0, row=4, sticky=W)
        ttk.Label(xyframe, textvariable=self.xt_txt, 
                  font=self.TFont1,foreground='teal').grid(column=1, row=4, sticky=W)
        ttk.Label(xyframe, text="Y robot: ", 
                  font=self.TFont1,foreground='maroon').grid(column=0, row=5, sticky=W)
        ttk.Label(xyframe, textvariable=self.yt_txt, 
                  font=self.TFont1,foreground='teal').grid(column=1, row=5, sticky=W)
        ttk.Label(xyframe, text="Theta : ", 
                  font=self.TFont1,foreground='maroon').grid(column=0, row=6, sticky=W)
        
        ttk.Label(xyframe, textvariable=self.theta_txt, 
                  font=self.TFont1,foreground='orangered').grid(column=1, row=6, sticky=W)
        ttk.Button(xyframe, text="Reset", 
                   command=self.send_cmd_reset).grid(column=0, 
                        row=7, sticky=W)
        
        
        # enable function would do the job ?
#         ttk.Button(xyframe, text="Move", command=self.send_goal).grid(column=0, 
#                                                         row=6, sticky=W) 

        # ------- draw path control ------
        self.drawmode = BooleanVar()
#         self.showws = BooleanVar()
        self.showhist = BooleanVar()
        drawframe = ttk.Frame(mainframe, padding = (2,2,2,2),relief="sunken")
        drawframe.grid(column=cvcolspan,row=4,sticky=(W))
        ttk.Label(drawframe, text="Plot control         ", 
                  font=self.TFont1,foreground='olive').grid(column=0, row=0, sticky=W)
        ttk.Checkbutton(drawframe, text="draw path",variable=self.drawmode).grid(column=0, 
                                                                row=1, sticky=W)
        ttk.Button(drawframe, text="Clear", command=self.clear_path).grid(column=0, 
                                     row=2, sticky=W)        
#         ttk.Checkbutton(drawframe, text="show workspace",variable=self.showws).grid(column=0, 
#                                     row=3, sticky=W)
        ttk.Checkbutton(drawframe, text="show history",variable=self.showhist).grid(column=0, 
                                    row=4, sticky=W)

        self.pauserb = (ttk.Radiobutton(drawframe,text="Pause", 
                                        variable=self.streaming_txt,
                                      value="0",
                command=self.send_cmd_streaming).grid(column=0,
                                    row=5, sticky=W))
        self.streamingrb = (ttk.Radiobutton(drawframe, text="Streaming", 
                                            variable=self.streaming_txt,
                                     value="1",
                            command=self.send_cmd_streaming).grid(column=0,
                                    row=6, sticky=W))
        ttk.Button(drawframe, text="Flush input", 
                   command=self.flush_input).grid(column=0, 
                        row=7, sticky=W)
        ttk.Button(drawframe, text="Flush output", 
                   command=self.flush_output).grid(column=1, 
                        row=7, sticky=W)
        self.usevsp = BooleanVar()
        self.usevsp.set(str(self.vspon))
        ttk.Checkbutton(drawframe, text="use VSP",
            variable=self.usevsp,command=self.use_vsp).grid(column=0, 
            row=8, sticky=W)
        
        ttk.Label(mainframe, text="dew.ninja 2024", font=self.TFont1, 
                  foreground="darkred").grid(column=cvcolspan,row=5,sticky=W)
        
        # ------ canvas for differential-drive robot display  ----------
        self.x_vec = []  # used for path draw
        self.y_vec = []
        self.xg_vec = []
        self.yg_vec = []
        self.fig, self.ax = plt.subplots(figsize=(9,9))
        cif = self.fig.canvas.mpl_connect('button_press_event',self.mouse_event) 

#         self.point_of_rotation = np.array([0, self.link_height/2])         
        # -----------------------------------------
        
#         self.time_step = 0.0
#         self.time_vec = []
#         self.y1_vec = []
#         self.y2_vec = []
#         self.y3_vec = []
       
#         self.fig, self.axes  = plt.subplots(2,1)
#         self.ax1, self.ax2 = self.fig.get_axes()
        self.canvas = FigureCanvasTkAgg(self.fig, master=mainframe)
        self.canvas.get_tk_widget().grid(row=0,column=0, rowspan=cvrowspan, columnspan=cvcolspan, sticky=W)
        self.an1 = FuncAnimation(self.fig, self.animate, interval=20,cache_frame_data=False )
#      save_count=MAX_FRAMES or cache_frame_data=False
        

        for child in mainframe.winfo_children(): 
            child.grid_configure(padx=5, pady=5)
        
#         self.Client_ID = "" # put in your NETPIE info 
#         self.User = "
#         self.Password = ""    
#         self.broker = "broker.netpie.io"
#         self.port = 1883     
#         if self.online:
#             self.mqtt_connect()

        self.enable = 0  # will be updated by ESP32
        self.streaming = 0
        self.streaming_txt.set(str(self.streaming))
        
        self.xg = 0.0  # goal coordinate. Start at (0.0,2.0)
        self.yg = 2.0
        self.xt = 0.0   # tool coordinate
        self.yt = 0.0
        self.theta = 0.0  # orientation

        self.enable_txt.set(str(self.enable))
        self.xg_txt.set(str(self.xg))
        self.yg_txt.set(str(self.yg))
        self.xt_txt.set(str(self.xt))
        self.yt_txt.set(str(self.yt))
        self.theta_txt.set(str(self.theta))
        
        
        # robot parameters
        self.LEFT_WHEEL = 0
        self.RIGHT_WHEEL = 1
        self.WHEEL_RADIUS = 0.08
        self.WHEEL_DIAMETER = 2*self.WHEEL_RADIUS
        self.WHEEL_THICKNESS = 0.04
        self.WHEEL_SEPARATION = 0.45
        self.BASE_LENGTH = 0.65
        self.BASE_WIDTH = 0.55
        self.EYE_THICKNESS = 0.04
        self.EYE_DIAMETER = 0.1
        

        
#         self.xg_txt.set(str(round(self.xg,2)))
#         self.yg_txt.set(str(round(self.yg,2)))
        

#     def mqtt_connect(self):
#         print("Connecting to mqtt broker")
#         try:
#             self.client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2,
#                         client_id=self.Client_ID,
#                          transport='tcp',
#                          protocol=mqtt.MQTTv311,
#                          clean_session=True)
#             self.client.username_pw_set(self.User,self.Password)
#             self.client.connect(self.broker, port=self.port, keepalive=60)
#             self.client.subscribe("@msg/update")
#             #self.client.subscribe("@msg/shadowdata")
#             self.client.on_message = self.on_message
#             self.client.loop_start()
#         except:
#             print("Problem connecting to mqtt broker")

#     def on_message(self,client, userdata, message):
#         self.rcvd_msg = str(message.payload.decode("utf-8"))
#         self.rcvd_topic = message.topic
# #         #print("message received ", self.rcvd_msg)
# #         #print("message topic=",message.topic)
#         self.xt_txt = StringVar()  # XY coordinate of robot
#         self.yt_txt = StringVar()
#         self.theta_txt = StringVar() # orientation of robot

#         if self.rcvd_topic == "@msg/update":
#             parm_data = self.rcvd_msg.split(',')
#             # strings for GUI update
#             self.enable_txt.set(parm_data[3])
#             self.xt_txt.set(parm_data[0])
#             self.yt_txt.set(parm_data[1])
#             self.theta_txt.set(parm_data[2])
            
#             # update parameters
#             self.enable = int(parm_data[3])
#             self.trackmode = int(parm_data[4])
#             if self.trackmode:
#                 self.trackmsg_txt.set("Tracking in progress")
#             else:
#                 self.trackmsg_txt.set("Ready")
                
#             self.xt = float(parm_data[0])
#             self.yt = float(parm_data[1])
#             self.theta = float(parm_data[2])

       
            
    def animate(self,i):
        # get data from VSP instead of IoT
        if self.vspon:
            # read data from serial buffer. Then loop to put all data points to plot vectors.
            if self.streaming:  
                self.msgline = self.ser.readline()
                datastr = self.msgline.decode().strip()
                if '$' not in datastr:  # not contaminated by system command/ response 
                    ddr_data = datastr.split(',')
                    #print(ddr_data)
                    #print(len(ddr_data))
                    if len(ddr_data)==5:  # this prevents read error 
                        self.xt = float(ddr_data[0])
                        self.yt = float(ddr_data[1])
                        self.theta = float(ddr_data[2])
                        self.enable = int(ddr_data[3])
                        self.trackmode = int(ddr_data[4])
                        
                        # set text for display purpose
                        self.xt_txt.set(ddr_data[0])
                        self.yt_txt.set(ddr_data[1])
                        self.theta_txt.set(ddr_data[2])
                        self.enable_txt.set(ddr_data[3])
                        if self.trackmode:
                            self.trackmsg_txt.set("Tracking in progress")
                        else:
                            self.trackmsg_txt.set("Ready")
        
        
        if not self.showhist.get():  # delete previous plot or not
            self.ax.cla()
        self.ax.set_aspect('equal')
        self.ax.set_xlim(-3.0,3.0)
        self.ax.set_ylim(-3.0,3.0)
        self.ax.set_xlabel("X")
        self.ax.set_ylabel("Y")

#         self.ax.plot(0, 0,  color='orange', marker='o', markersize=30)
        base_height = self.BASE_WIDTH
        base_width = self.BASE_LENGTH
#         fig, ax = plt.subplots(figsize=(10,10))
        #ax.plot(0, 0,  color='b', marker='o', markersize=30)

        # find 5 points of robot related to theta
        # center, left-right wheels, left-right eyes (front size)
        x_center = self.xt
        y_center = self.yt
        theta = self.theta*self.rad2deg
#         print("{},{},{}".format(x_center,y_center,theta))


        robot_point_of_rotation = np.array([base_width/2, base_height/2]) # mid point

        robot_base = RotatingRectangle((x_center,y_center), width=base_width, height=base_height, 
                                    rel_point_of_rot=robot_point_of_rotation,
                                    angle=theta, edgecolor='black', facecolor='orange')
        self.ax.add_patch(robot_base)

        # left wheel 
        x_lw = 0
        y_lw = self.WHEEL_SEPARATION/2

        # right wheel
        x_rw = 0
        y_rw = - self.WHEEL_SEPARATION/2

        # left eye
        x_le = self.BASE_LENGTH/2
        y_le = self.BASE_WIDTH/4

        # right eye
        x_re = self.BASE_LENGTH/2
        y_re = -self.BASE_WIDTH/4

        # rotate these points relative to theta
        xy_lw = self.rotate_points(theta,(x_lw,y_lw),(x_center,y_center))
        xy_rw = self.rotate_points(theta,(x_rw,y_rw),(x_center,y_center))
        xy_le = self.rotate_points(theta,(x_le,y_le),(x_center,y_center))
        xy_re = self.rotate_points(theta,(x_re,y_re),(x_center,y_center))
        # xy_lw = (x_lw,y_lw)
        # xy_rw = (x_rw,y_rw)
        # xy_le = (x_le,y_le)
        # xy_re = (x_re,y_re)
        wheel_height = self.WHEEL_THICKNESS
        wheel_width = self.WHEEL_DIAMETER

        # left wheel
        lw_point_of_rotation = np.array([wheel_width/2, wheel_height/2]) # mid point of left wheel
        robot_lw = RotatingRectangle(xy_lw, width=wheel_width, height=wheel_height, 
                                    rel_point_of_rot=lw_point_of_rotation,
                                    angle=theta, edgecolor='black', facecolor='blue')
        self.ax.add_patch(robot_lw)

        # right wheel
        rw_point_of_rotation = np.array([wheel_width/2, wheel_height/2]) # mid point of right wheel
        robot_rw = RotatingRectangle(xy_rw, width=wheel_width, height=wheel_height, 
                                    rel_point_of_rot=rw_point_of_rotation,
                                    angle=theta, edgecolor='black', facecolor='blue')
        self.ax.add_patch(robot_rw)

        # ---- add eyes ---------
        eye_height = self.EYE_DIAMETER
        eye_width = self.EYE_THICKNESS

        # left eye
        le_point_of_rotation = np.array([eye_width/2, eye_height/2]) # mid point of left eye
        robot_le = RotatingRectangle(xy_le, width=eye_width, height=eye_height, 
                                    rel_point_of_rot=le_point_of_rotation,
                                    angle=theta, edgecolor='black', facecolor='red')
        self.ax.add_patch(robot_le)

        # right eye
        re_point_of_rotation = np.array([eye_width/2, eye_height/2]) # mid point of left eye
        robot_re = RotatingRectangle(xy_re, width=eye_width, height=eye_height, 
                                    rel_point_of_rot=re_point_of_rotation,
                                    angle=theta, edgecolor='black', facecolor='red')
        self.ax.add_patch(robot_re)

        self.ax.grid(True)


        self.ax.plot(self.xg, self.yg,  color='r', marker='X', markersize=15) # target position

#         if self.showws.get(): # draw workspace
#             robo_ws = plt.Circle((0.0,0.0),2, color='m',fill=False)
#             self.ax.add_patch(robo_ws)
            
        if self.drawmode.get():   # draw path
            self.x_vec.append(self.xt)
            self.y_vec.append(self.yt)
            self.ax.plot(self.x_vec,self.y_vec,'blue')
            self.ax.plot(self.xg_vec,self.yg_vec,'red',marker='X',linestyle='None',markersize=15)
#        print("Animating")


    def send_ddr(self, *args):  # changed to send arbitrary command
        cmd_txt = "$"+self.ddr_txt.get()+"\r\n"
        if self.vspon:
            self.ser.write(cmd_txt.encode('utf-8'))
        
    def send_cmd_disable(self, *args):
        if self.vspon:
            cmd_txt = "$enable=0\r\n"
            self.ser.write(cmd_txt.encode('utf-8'))
    def send_cmd_enable(self, *args):
        if self.vspon:
            cmd_txt = "$enable=1\r\n"
            self.ser.write(cmd_txt.encode('utf-8'))
    def send_cmd_reset(self, *args):
        if self.vspon:
            cmd_txt = "$reset\r\n"
            self.ser.write(cmd_txt.encode('utf-8'))

    def send_goal(self, *args): # send step or track command to robot controller
        if self.vspon:
            cmd_txt = "$ltrack="+str(round(self.xg,3))+","+str(round(self.yg,3))+"\r\n"
            self.ser.write(cmd_txt.encode('utf-8'))
            

    def clear_path(self, *args):         
        self.x_vec = []
        self.y_vec = []
        self.xg_vec = []
        self.yg_vec = []
#     def show_workspace(self, *args):         
#         pass

    def mouse_event(self,event):
        self.xg = round(event.xdata,3)
        self.yg = round(event.ydata,5)
        self.xg_txt.set(str(self.xg))
        self.yg_txt.set(str(self.yg))
        if self.drawmode.get():   # draw path
            self.xg_vec.append(self.xg)
            self.yg_vec.append(self.yg)        
        self.send_goal()
#         self.ikine()

    def rotate_points(self,angle,xy,center):
        angle_rad = angle * np.pi / 180
        m_trans = np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                        [np.sin(angle_rad), np.cos(angle_rad)]])
        shift = m_trans @ xy    
        return np.add(shift,center)
    
    def flush_input(self, *args):   
        if self.vspon:                               
            self.ser.reset_input_buffer()
    def flush_output(self, *args): 
        if self.vspon:                               
            self.ser.reset_output_buffer()
    def use_vsp(self, *args): 
        self.vspon = self.usevsp.get()
        if self.vspon:                               
            self.ser = serial.Serial('COM8', 115200, timeout=1)
            self.ser.reset_input_buffer() # flush all data that you don't wanna read
            self.ser.reset_output_buffer()
        else:
            self.ser.close()
            
    def send_cmd_streaming(self, *args):
        if self.vspon: 
            self.streaming = int(self.streaming_txt.get())
            cmd_txt = "$streaming="+self.streaming_txt.get()+"\r\n"
            self.ser.write(cmd_txt.encode('utf-8'))    
            time.sleep(5)
            self.ser.reset_input_buffer()
            self.ser.reset_output_buffer()


            if self.streaming:
                self.an1.resume()
            else:
                self.an1.pause()
                
    def send_cmd_vk(self, *args):
        if self.vspon:        
            cmd_txt = "$vk="+self.vk_txt.get()+"\r\n"
            self.ser.write(cmd_txt.encode('utf-8'))
    def send_cmd_wk(self, *args):
        if self.vspon:        
            cmd_txt = "$wk="+self.wk_txt.get()+"\r\n"
            self.ser.write(cmd_txt.encode('utf-8'))

        
    def on_close(self):
        print("Cleaning up.. ")
        if self.vspon:
            self.ser.close()
        root.destroy()

root = Tk()
ddrControllerVSP(root)
root.mainloop()
    